# 3. Composite types 

In the previous notebook we began to see the need for a way to be able to describe objects that we are modelling in code, in such a way that the objects can have internal properties which are grouped together in one place.

Our main goal will be to model a person who can move *and* has an infection status, telling us whether they are susceptible, infectious or recovered. We could also add other information, such as how many times it has passed through 0 or which direction it moved on the previous step.

## The main idea

The main **idea** is to collect up or **aggregate** all relevant information into a new data structure, called a **composite type** (or custom type, aggregate type, user-defined type, ...).

Basically we want to be able to specify the "template" / "shape" / "structure" for a bag or box that will contain all the relevant information; this specification is the type itself. Then we need to produce objects which have that structure, i.e. which *contain* the corresponding variables; these are called **instances**.

In Julia this is accomplished using the `struct` keyword (short for "structure"). For example, we can make an object that contains the $x$ and $y$ coordinates of a walker in 2 dimensions as

In [ ]:
struct Walker2D
    x::Int64
    y::Int64
end

What is `Walker2D`? It is the name of a new *type*:

In [ ]:
Walker2D

It is also the name of a pair of functions that Julia has automatically created for us:

In [ ]:
methods(Walker2D)

These functions are used to create or construct objects of this type; they are called **constructors**. For example:

In [ ]:
w = Walker2D(1, 2)

This has created an object `w`, of type `Walker2D`. 

In [ ]:
w

In [ ]:
typeof(w)

The `x` and `y` coordinates of the walker are the **fields**. They live inside the object and can be accessed using the `.` operator:

In [ ]:
w.x

In [ ]:
w.y

We can create another walker by calling the constructor again:

In [ ]:
w2 = Walker2D(3, 4)

It's fields are different:

In [ ]:
w2.x

In [ ]:
w2.y

Let's verify that `w`s data is still the same as it was:

In [ ]:
w.x, w.y

So we see that this mechanism successfully allows us to create *different* objects of the same type. We can really now talk about having "two separate walkers" in our code, in a way that accurately reflects the situation we are trying to model.

## (Outer) constructors

Suppose we want walkers to be born at the origin unless otherwise stated. We don't want to have to write `Walker2D(0, 0)` each time; we would like to just write `Walker2D()`. In other words, we want to *add a new method* to the function `Walker2D`:

In [ ]:
Walker2D() = Walker2D(0, 0)

We can see that indeed a new method was created using the `methods` function:

In [ ]:
methods(Walker2D)

Such a constructor is called an **outer constructor**, since it lives *outside* the definition of the type. [There are also inner constructors, which are used *only* if, for example, you want to be able to prevent creating objects which are invalid under some criterion. Prefer outer constructors whenever possible.]


## Making walkers move

Now we would like to make a walker move. We might think this would be easy: we should just modify one of its fields. However:

In [ ]:
w.x = 10

[`setfield!` is the function that is called when we try to change a field.]

We are not allowed to modify the fields because we defined `Walker2D` as an **immutable** `struct`. 
"Mutation" is a computer science term, referring to the ability to modify an object. `struct`s are immutable (non-modifiable) in Julia by default. Usually you will have better performance if you are able to use immutable `struct`s. [Technically, they are probably stored on the stack, instead of the heap.]

So in order to make our walker move, we need to create a *new* object with the new position. This could seem expensive, but in fact the Julia compiler will often be able to completely remove this object creation and produce code that is just as efficient as if there were no object at all!

#### Exercise
1. Make a function `move_right` that takes a `Walker2D` as argument. It returns a new walker, located one step to the right.

2. Make a function `jump` that takes a `Walker2D` as argument and makes it move to a random adjacent location. You should choose either the $x$ or $y$ coordinate at random, and move that coordinarte by 1 step in a random direction.

In Julia, variables are names that are associated to values, i.e. they are **bindings** that "point to" objects located in memory. [Strictly speaking, the object may not actually have a physical location in memory; it may just live inside the CPU's registers. In fact, this is the best situation for performance.]

We can update the binding by assigning a new object to that variable name:

In [ ]:
w = Walker2D(1, 2)
w = move_right(w)   # should return (2, 2)

In this way we can have variables that refer to immutable `struct`s but whose value we can change.

## Generic programming with types

Let's think about our `walk` function from the previous notebook:

In [ ]:
function walk(T)
    pos = initialize()
    
    trajectory = [pos]    # make a vector that contains one item, the current value of `pos`
    
    for t in 1:T
        pos = move(pos)
        push!(trajectory, deepcopy(pos))
    end
    
    return trajectory
end

In the previous notebook we changed the behaviour by passing in different functions as arguments. Now, however, we have a (usually) better solution: we can make objects of different *types* behave in different ways! 

Indeed, we have not actually said what **types** are. They are basically labels that tell Julia how data (i.e. bits in memory) should **behave** when acted on by different functions. For example, internally Julia must call *different* functions to add two integers or two floating-point numbers together, since the internal representation of these numbers is very different.

So let's rewrite `walk` so that it acts on an *object* `w` representing our walker:

In [ ]:
"Calculate the trajectory of a walker `w` for time `T`."
function walk(w, T)
    
    trajectory = [w]   # store the current (initial) position of `w`
    
    for t in 1:T
        w = move(w)    # update the value bound to `w`
        push!(trajectory, deepcopy(w))   # store the current value
    end
    
    return trajectory
end

It now accepts an argument `w`, which we expect to be some type of walker object. Note that we no longer call `initialise` &ndash; rather, we require that the user create their favourite type of walker object and pass it to us as an additional *input argument* to the function.

What makes something a "walker object"? If we look at the function, we see that the only thing we actually do to `w` is to call `move` on it. So we have an informal **interface**: *anything* with a function `move` defined (that behaves in the correct way) will be OK! This is another example of generic programming.

For example, we can use our function with standard Julia integers by defining a `walk` function on them:

In [ ]:
move(w::Integer) = w + 1

Here, `w::Integer` is a **type annotation**. It is saying "this version of the `walk` function applies when the argument is of type `Integer`. (Note that `Integer` is an abstract type that includes all concrete integer types, such as `Int64` and `BigInt`.) 

Versions of functions (with the *same* function name) are known as **methods** in Julia. A (generic) function is made up of a patchwork of different methods that act on objects of different types. When we add a method that acts on a new type or combination of types we talk about **extending** the function.

Note that it is rare for functions to live inside types in Julia, unlike in object-oriented languages. Functions are too important to be hidden away inside types! 
This is one of the main things that gives Julia a very different flavour and makes it much more flexible for scientific / technical computing.

Now we can walk down the integers:

In [ ]:
x0 = 3  # initial position
T = 10  # length of trajectory

trajectory = walk(x0, T)'

#### Exercise

1. Define a `Walker1D` type and a *new method* for the `move` function (with the *same* function name, `move`!).
Plot some trajectories as a function of time.


2. Use the `Walker2D` type, together with the random jump code from the last notebook, to define another new method for `move`.


3. Use this to calculate and plot a trajectory in space. Hint: An array comprehension is a simple way of extracting a given field from an object.


4. Make an interactive visualization of a cloud of walkers over time.

#### Exercise

1. Make a `CorrelatedWalker1D` type representing a 1-dimensional **correlated random walk**. This is a random walk that remembers its previous step direction &ndash; In the language of computer science, it **has state**, or is **stateful**.
The walker takes the following step in the *same* direction with probability $p$, and in the opposite direction with probability $1-p$. The parameter $p$ and the direction must thus be additional fields of the object.


2. Calculate and draw some trajectories.

## Mutable `struct`s

An alternative approach is to use *mutable* structs, where the fields *can* be modified. Functions that modify the content of an object have `!` appended to their names by convention:

In [ ]:
mutable struct MutableWalker1D
    x::Int
end

In [ ]:
w = MutableWalker1D(0)

In [ ]:
w.x = 1   # now this works!

However, it is usually preferable for getting good performance to use immutable `struct`s.

#### Exercise
1. Define a `move!` function that updates the value stored inside the `MutableWalker1D` object. Can you re-use code you have already written?